In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from patsy import dmatrices

In [5]:
dta = pd.read_csv('../data/dayData.csv')

In [9]:
dta.describe()

,household_key,DAY,QUANTITY,BASE_SPEND_AMT,NET_SPEND_AMT,LOY_CARD_DISC,COUPON_DISC,GET_EGGS
count,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000,25797.000000
mean,505.790402,101.607086,14.598752,42.941644,36.737954,-6.060541,-0.143149,0.146063
std,292.033469,58.799244,17.332974,51.432689,43.380989,8.682502,0.921875,0.367696
min,1.000000,0.000000,1.000000,0.210000,0.210000,-208.020000,-43.280000,0.000000
25%,257.000000,51.000000,3.000000,10.060000,8.970000,-7.830000,0.000000,0.000000
50%,511.000000,102.000000,8.000000,24.190000,21.010000,-2.970000,0.000000,0.000000
75%,761.000000,152.000000,19.000000,55.730000,47.560000,-0.790000,0.000000,0.000000
max,1000.000000,204.000000,261.000000,816.790000,681.650000,0.000000,0.000000,3.000000


In [8]:
dta = dta.drop('Unnamed: 0', 1)

In [21]:
y, X = dmatrices('GET_EGGS ~ DAY + QUANTITY + BASE_SPEND_AMT + NET_SPEND_AMT + LOY_CARD_DISC + COUPON_DISC', dta, return_type="dataframe")

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [120]:
predicted = model.predict(X_test)
print(predicted)

[ 0.  0.  0. ...,  0.  0.  0.]


In [49]:
probs = model.predict_proba(X_test)
print(probs)

[[  9.06108658e-01   9.06275074e-02   3.22083016e-03   4.30044867e-05]
 [  9.02909100e-01   9.25835670e-02   3.98556428e-03   5.21768985e-04]
 [  9.34581238e-01   6.19655506e-02   2.93718244e-03   5.16028743e-04]
 ..., 
 [  9.15034667e-01   8.09995977e-02   3.38595095e-03   5.79784829e-04]
 [  9.11912632e-01   8.47456278e-02   3.27385264e-03   6.78873830e-05]
 [  9.32026272e-01   6.48839839e-02   2.89278984e-03   1.96954347e-04]]


In [121]:
metrics.brier_score_loss(y_test, predicted)

0.034496124031007755

In [94]:
print(y_train.sum(axis=0)[0])
print(y_train.shape[0])
freq = y_train.sum(axis=0)[0] / y_train.shape[0]
print(y_test.shape[0])
predicted = [0.0001 for idx in range(0, y_test.shape[0])]

2633.0
18057
7740


In [101]:
y_test['GET_EGGS'].value_counts()

0.0    6643
1.0    1061
2.0      34
3.0       2
Name: GET_EGGS, dtype: int64

In [111]:
np.count_nonzero(predicted==1)

267

In [115]:
model2 = RandomForestClassifier()
model2.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [118]:
prediction2 = model2.predict(X_test)
print(prediction2)

[ 0.  0.  0. ...,  0.  0.  0.]


In [119]:
metrics.brier_score_loss(y_test, prediction2)

0.050775193798449615